In [11]:
import pynvml as pyn
from pynvml.smi import nvidia_smi
nvml = nvidia_smi.getInstance()
gpu = pyn.nvmlDeviceGetHandleByIndex(0)

In [28]:
Gib = 1073741824
info = pyn.nvmlDeviceGetMemoryInfo(gpu)
print(info.total/Gib)

16.0


In [35]:
temp = pyn.nvmlDeviceGetTemperature(gpu, pyn.NVML_TEMPERATURE_GPU)
temp

33